In [1]:
import numpy as np
import scipy.sparse as sparse

In [2]:
def record_output(file: str, vars: list):
    f = open(file, 'a')
    t = ','.join(np.array(vars).astype(str)) + '\n'
    f.write(t)
    f.close()

In [3]:
# Number of rows and columns
N_i, N_j = 10, 1
epsilon = 0
J = 1
BC_type = 'PBC'

pauli_z = sparse.diags_array([1, -1])

In [4]:
def n_identities(n: int):
    """Gives a tensor made out of the tensor product on 'n' identity matrices.

    Args:
        n (int): the number of identity matrices

    Returns:
        np.ndarray: the resultant matrix (tensor)
    """
    return sparse.identity(2**n) if (n > 0) else 1

$$\mathbf{A}_{i j} = \underbrace{\overbrace{\mathbf{I} \otimes \ldots \otimes\mathbf{I}}^{N_j i + j\ \text{operators}} \otimes \sigma^z \otimes \overbrace{\mathbf{I} \otimes \ldots \otimes\mathbf{I}}^{N_j N_i - N_j i - j - 1\ \text{operators}}}_{N_i N_j\ \text{operators}}, \quad i,j = 0, 1, 2, \ldots$$

In [5]:
# A list to store the operators A
A_ops = np.empty((N_i,N_j), dtype=sparse.coo_matrix)

for i in range(N_i):
    for j in range(N_j):
        A_ij = sparse.kron(sparse.kron(n_identities((N_j*i)+j),
                                       pauli_z),
                           n_identities((N_j*N_i)-(N_j*i)-j-1))
        A_ops[i,j] = A_ij

$$\mathbf{H} = \epsilon \sum_{i, j = 0}^{N_i - 1, N_j - 1} \mathbf{A}_{i j} + \frac{1}{2} J \sum_{i, j = 0}^{N_i - 1, N_j - 1} \mathbf{A}_{i j} (\mathbf{A}_{i + 1, j} + \mathbf{A}_{i - 1, j} + \mathbf{A}_{i, j + 1} + \mathbf{A}_{i, j - 1})$$

In [6]:
# Creating the Hamiltonian
H = sparse.dia_matrix((2**(N_i*N_j), 2**(N_i*N_j)), dtype=float)

match BC_type:
    case 'PBC':
        for i in range(N_i):
            for j in range(N_j):
                H += epsilon * A_ops[i,j]
                
                # Neighbors on the same row
                for index in [j+1, j-1]:
                    # If index is (i, -1), there will be no problem
                    try:
                        H += 0.5 * J * (A_ops[i,j] @ A_ops[i, index])
                    # IndexError means that the index is out of bound,
                    # so index 0 should be used.
                    except IndexError:
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[i, 0])
                        except:
                            raise Exception('failed to find element \
                            ({:.0f},{:.0f}) in A_ops'.format(i, index))

                # Neighbors on the same column
                for index in [i+1, i-1]:
                    # If index is (-1, j), there will be no problem
                    try:
                        H += 0.5 * J * (A_ops[i,j] @ A_ops[index, j])
                    # IndexError means that the index is out of bound,
                    # so index 0 should be used.
                    except IndexError:
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[0, j])
                        except:
                            raise Exception('failed to find element \
                            ({:.0f},{:.0f}) in A_ops'.format(index, j))

    case 'APBC':
        for i in range(N_i):
            for j in range(N_j):
                H += epsilon * A_ops[i,j]
                
                # Neighbors on the same row
                for index in [j+1, j-1]:
                    # If index is (i, -1), there will be no problem
                    try:
                        if index == -1:
                            H += 0.5 * J * (A_ops[i,j] @ (-A_ops[i, -1]))
                        else:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[i, index])
                    # IndexError means that the index is out of bound,
                    # so index 0 should be used.
                    except IndexError:
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ (-A_ops[i, 0]))
                        except:
                            raise Exception('failed to find element \
                            ({:.0f},{:.0f}) in A_ops'.format(i, index))
    
                # Neighbors on the same column
                for index in [i+1, i-1]:
                    # If index is (-1, j), there will be no problem
                    try:
                        if index == -1:
                            H += 0.5 * J * (A_ops[i,j] @ (-A_ops[-1, j]))
                        else:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[index, j])
                    # IndexError means that the index is out of bound,
                    # so index 0 should be used.
                    except IndexError:
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ (-A_ops[0, j]))
                        except:
                            raise Exception('failed to find element \
                            ({:.0f},{:.0f}) in A_ops'.format(index, j))

    case default:
        raise Exception('BC_type is not correctly defined')

print("Hamiltonian =\n", H)

Hamiltonian =
   (0, 0)	20.0
  (1, 1)	16.0
  (2, 2)	16.0
  (3, 3)	16.0
  (4, 4)	16.0
  (5, 5)	12.0
  (6, 6)	16.0
  (7, 7)	16.0
  (8, 8)	16.0
  (9, 9)	12.0
  (10, 10)	12.0
  (11, 11)	12.0
  (12, 12)	16.0
  (13, 13)	12.0
  (14, 14)	16.0
  (15, 15)	16.0
  (16, 16)	16.0
  (17, 17)	12.0
  (18, 18)	12.0
  (19, 19)	12.0
  (20, 20)	12.0
  (21, 21)	8.0
  (22, 22)	12.0
  (23, 23)	12.0
  (24, 24)	16.0
  :	:
  (999, 999)	16.0
  (1000, 1000)	12.0
  (1001, 1001)	12.0
  (1002, 1002)	8.0
  (1003, 1003)	12.0
  (1004, 1004)	12.0
  (1005, 1005)	12.0
  (1006, 1006)	12.0
  (1007, 1007)	16.0
  (1008, 1008)	16.0
  (1009, 1009)	16.0
  (1010, 1010)	12.0
  (1011, 1011)	16.0
  (1012, 1012)	12.0
  (1013, 1013)	12.0
  (1014, 1014)	12.0
  (1015, 1015)	16.0
  (1016, 1016)	16.0
  (1017, 1017)	16.0
  (1018, 1018)	12.0
  (1019, 1019)	16.0
  (1020, 1020)	16.0
  (1021, 1021)	16.0
  (1022, 1022)	16.0
  (1023, 1023)	20.0


In [7]:
# Getting the smallest eigenvalues and their corresponding eigenvectors
eig_vals, eig_vecs = sparse.linalg.eigs(H, which='SR')

# Turning (2**N, 6) eig_vecs into (6, 2**N)
eig_vecs = eig_vecs.T
# Removing unnecessary computed values
eig_vecs[abs(eig_vecs) < 1e-10] = 0
eig_vals = np.round(eig_vals, decimals=10)

# The lowest energy
E_0 = np.real(np.min(eig_vals))
# The state with the lowest energy
psi_0 = eig_vecs[eig_vals == E_0].T

psi_0_row, psi_0_col = psi_0.shape
# Degeneracy handler
if psi_0_col > 1:
    # Setting the first state as the ground state
    psi_0 = psi_0[:,0].reshape(-1, 1)
    # scipy.sparse.linalg.eigs gives only 6 eigenvalues.
    #The degeneracy can be more than that.
    print("E_0 degeneracy =\n", psi_0_col, "(at least)")

print("E_0 =\n", E_0)

E_0 degeneracy =
 2 (at least)
E_0 =
 0.0


$$\mathbf{M} = \sum_{i, j = 1}^{N} \mathbf{A}_{i j}, \quad M_0 = \left< \psi_0 \right| \mathbf{M} \left| \psi_0 \right>$$

In [8]:
# Magnetization operator
M_op = A_ops.sum()
M = psi_0.conjugate().T @ M_op @ psi_0
if M.shape == (1, 1):
    M = np.real(M[0,0])

print("M =\n", M)

M =
 0.0
